## Limpieza

In [56]:
import pandas as pd
import numpy as np
import re

In [57]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)

In [58]:
df = pd.read_csv('../data/raw/hr.csv')
df.head(3)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41.0,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,2,Female,94,3,2,sALES eXECUTIVE,4.0,Single,5993.0,19479,8,Y,Yes,11,3,1,80.0,0,8,0.0,1,6,4,0,5.0
1,49.0,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,3,Male,61,2,2,rESEARCH sCIENTIST,2.0,Married,5130.0,24907,1,Y,No,23,4,4,NaN,1,10,3.0,3,10,7,1,7.0
2,37.0,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,4,Male,92,2,1,lABORATORY tECHNICIAN,3.0,Single,2090.0,2396,6,Y,Yes,15,3,2,NaN,0,7,3.0,3,0,0,0,0.0


In [59]:
def normalizar_nombres_columnas(lista_columnas, mostrar_resumen=True):
    """
    Normaliza una lista de nombres de columnas de DataFrame.
    
    Esta normalización incluye:
    - Eliminación de espacios al inicio y al final
    - Eliminación de caracteres especiales (excepto guiones bajos)
    - Conversión de CamelCase o PascalCase a snake_case
    - Conversión de todos los caracteres a minúsculas
    
    Parámetros:
    - lista_columnas: lista de strings con nombres de columnas originales
    - mostrar_resumen: bool, si True imprime un resumen de los cambios
    
    Retorna:
    - lista de nombres de columnas normalizados en snake_case
    """
    nombres_normalizados = []
    
    for nombre in lista_columnas:
        limpia = nombre.strip()  # quitar espacios al inicio y final
        limpia = re.sub(r'[^0-9a-zA-Z_]', '', limpia)  # eliminar caracteres especiales salvo guion bajo
        limpia = re.sub(r'([a-z0-9])([A-Z])', r'\1_\2', limpia) # insertar guion bajo entre minúscula/número y mayúscula
        nombres_normalizados.append(limpia.lower())  # convertir a minúsculas

    # Mostrar resumen opcional
    if mostrar_resumen:
        print("Normalización de nombres de columnas finalizada.")
        print(f"Total columnas procesadas: {len(nombres_normalizados)}")
        print("Resumen de cambios:")
        for orig, nuevo in zip(lista_columnas, nombres_normalizados):
            print(f"'{orig}' -> '{nuevo}'")
    
    return nombres_normalizados


In [60]:
df.columns = normalizar_nombres_columnas(df.columns.tolist(), mostrar_resumen=True)

Normalización de nombres de columnas finalizada.
Total columnas procesadas: 35
Resumen de cambios:
'Age' -> 'age'
'Attrition' -> 'attrition'
'BusinessTravel' -> 'business_travel'
'DailyRate' -> 'daily_rate'
'Department' -> 'department'
'DistanceFromHome' -> 'distance_from_home'
'Education' -> 'education'
'EducationField' -> 'education_field'
'EmployeeCount' -> 'employee_count'
'EmployeeNumber' -> 'employee_number'
'EnvironmentSatisfaction' -> 'environment_satisfaction'
'Gender' -> 'gender'
'HourlyRate' -> 'hourly_rate'
'JobInvolvement' -> 'job_involvement'
'JobLevel' -> 'job_level'
'JobRole' -> 'job_role'
'JobSatisfaction' -> 'job_satisfaction'
'MaritalStatus' -> 'marital_status'
'MonthlyIncome' -> 'monthly_income'
'MonthlyRate' -> 'monthly_rate'
'NumCompaniesWorked' -> 'num_companies_worked'
'Over18' -> 'over18'
'OverTime' -> 'over_time'
'PercentSalaryHike' -> 'percent_salary_hike'
'PerformanceRating' -> 'performance_rating'
'RelationshipSatisfaction' -> 'relationship_satisfaction'
'S

In [61]:
def usar_columna_como_indice(df, columna_original='employee_number', indice='id'):
    """
    Establece una columna del DataFrame como índice del mismo
    y la renombra como 'id' u otro nombre especificado.
    
    Parámetros:
    - df: DataFrame a modificar
    - columna_original: nombre de la columna a usar como índice
    - nuevo_nombre: nombre corto que se asignará al índice
    
    Retorna:
    - DataFrame con la columna establecida como índice y renombrada
    """
    
    df = df.copy()
    
    # Verificar que la columna exista
    if columna_original not in df.columns:
        raise ValueError(f"La columna '{columna_original}' no existe en el DataFrame.")
    
    # Renombrar columna
    df.rename(columns={columna_original: indice}, inplace=True)
    
    # Establecer como índice
    df.set_index(indice, inplace=True)
    
    print(f"Columna '{columna_original}' renombrada a '{indice}' y establecida como índice correctamente.")
    
    return df

In [62]:
df = usar_columna_como_indice(df, columna_original='employee_number', indice='id')

Columna 'employee_number' renombrada a 'id' y establecida como índice correctamente.


In [63]:
def eliminar_filas_duplicadas(df, keep='first'):
    """
    Elimina filas duplicadas del DataFrame.

    Parámetros:
    - df: DataFrame a procesar
    - keep: define qué duplicados conservar
        'first'  -> conserva la primera aparición
        'last'   -> conserva la última aparición
        False    -> elimina todas las filas duplicadas

    Retorna:
    - DataFrame sin filas duplicadas
    """
    df = df.copy()
    num_filas_antes_limpieza = df.shape[0]

    # Eliminar filas duplicadas según el parámetro 'keep'
    df.drop_duplicates(keep=keep, inplace=True)

    # Calcular cuántas filas se eliminaron
    filas_eliminadas = num_filas_antes_limpieza - df.shape[0]

    print(f"Filas antes de eliminar duplicados: {num_filas_antes_limpieza}")
    print(f"Filas eliminadas por duplicados: {filas_eliminadas}")
    print(f"Filas en el DataFrame tras el procesamiento: {df.shape[0]}")

    return df

In [64]:
df = eliminar_filas_duplicadas(df, keep='first')

Filas antes de eliminar duplicados: 1474
Filas eliminadas por duplicados: 4
Filas en el DataFrame tras el procesamiento: 1470


In [65]:
def eliminar_columnas_sin_aporte_analitico(df, umbral_cardinalidad=0.95):
    """
    Elimina columnas del DataFrame que no aportan valor analítico:
    - Columnas constantes (todos los valores iguales)
    - Columnas con alta cardinalidad (proporción de valores únicos mayor al umbral). 
    Se considera alta cardinalidad si es superior a 0.95

    Parámetros:
    - df: DataFrame a limpiar
    - umbral_cardinalidad: proporción máxima de valores únicos permitida (float entre 0 y 1)

    Retorna:
    - DataFrame con las columnas constantes o de alta cardinalidad eliminadas
    """
    
    df = df.copy()
    
    columnas_a_eliminar = []

    
    for columna in df.columns:
        # Columna constante
        if df[columna].nunique() == 1:
            columnas_a_eliminar.append(columna)
        # Columna con alta cardinalidad
        elif df[columna].nunique() / len(df) > umbral_cardinalidad:
            columnas_a_eliminar.append(columna)

    # Eliminar columnas identificadas
    df.drop(columns=columnas_a_eliminar, inplace=True)

    print(f"Columnas eliminadas por no aportar valor analítico: {columnas_a_eliminar}")
    print(f"Total columnas eliminadas: {len(columnas_a_eliminar)}")
    print(f"Columnas restantes en el DataFrame: {df.shape[1]}")

    return df

In [66]:
df = eliminar_columnas_sin_aporte_analitico(df, umbral_cardinalidad=0.95)

Columnas eliminadas por no aportar valor analítico: ['employee_count', 'monthly_rate', 'over18', 'standard_hours']
Total columnas eliminadas: 4
Columnas restantes en el DataFrame: 30


In [69]:
def normalizar_columnas_texto(df, mapeos_reemplazo=None, mostrar_resumen=True):
    """
    Normaliza columnas de texto o categóricas y aplica mapeos de reemplazo opcionales.

    Pasos de normalización:
    1. Elimina espacios al inicio y al final de cada valor.
    2. Convierte el texto a 'Title Case' (primera letra mayúscula, resto minúscula).
    3. Aplica mapeos de reemplazo si se proporciona un diccionario (para casos específicos).

    Parámetros:
    - df: DataFrame a limpiar (debe ser un pd.DataFrame)
    - mapeos_reemplazo: diccionario opcional con claves como nombres de columnas
      y valores como diccionarios de reemplazo {'valor_antiguo': 'valor_nuevo'}
    - mostrar_resumen: bool, si True imprime resumen de columnas procesadas y categorías

    Retorna:
    - DataFrame con columnas de texto normalizadas
    """

    df = df.copy()
    columnas_texto = df.select_dtypes(include=['object', 'category']).columns.tolist()
    columnas_con_reemplazo = []

    for col in columnas_texto:
        # Eliminar espacios y pasar a Title Case
        df[col] = df[col].str.strip().str.title()

        # Aplicar mapeos de reemplazo si existen
        if mapeos_reemplazo and col in mapeos_reemplazo:
            df[col] = df[col].replace(mapeos_reemplazo[col])
            columnas_con_reemplazo.append(col)

    # Mostrar resumen opcional
    if mostrar_resumen:
        print("Normalización de columnas de texto finalizada.")
        print(f"Columnas procesadas: {columnas_texto}")

        if columnas_con_reemplazo:
            print(f"Se aplicaron mapeos de reemplazo en: {columnas_con_reemplazo}")
            for col in columnas_con_reemplazo:
                print(f"Categorías finales de '{col}': {df[col].unique().tolist()}")

    return df

In [70]:
mapeos_reemplazo = {
    'marital_status': {'Marreid': 'Married'},  # corregir typo
    'business_travel': {
        'Travel_Rarely': 'Rarely',
        'Travel_Frequently': 'Frequently',
        'Non-Travel': 'Non'
    }
}

df = normalizar_columnas_texto(df, mapeos_reemplazo=mapeos_reemplazo)

Normalización de columnas de texto finalizada.
Columnas procesadas: ['attrition', 'business_travel', 'department', 'education_field', 'gender', 'job_role', 'marital_status', 'over_time']
Se aplicaron mapeos de reemplazo en: ['business_travel', 'marital_status']
Categorías finales de 'business_travel': ['Rarely', 'Frequently', 'Non', nan]
Categorías finales de 'marital_status': ['Single', 'Married', 'Divorced', nan]


In [73]:
def convertir_tipos_columnas(df, mapeo_tipos, mostrar_resumen=True):
    """
    Convierte columnas de un DataFrame a tipos de datos específicos.

    Parámetros:
    - df: pd.DataFrame a modificar
    - mapeo_tipos: diccionario con claves como nombres de columnas y
      valores como tipos de datos deseados (ej. int, float, 'Int64', 'category')
    - mostrar_resumen: bool, si True imprime un resumen de columnas convertidas

    Retorna:
    - DataFrame con columnas convertidas a los tipos especificados
    """
    df = df.copy()
    columnas_convertidas = []

    for columna, tipo in mapeo_tipos.items():
        if columna in df.columns:
            try:
                df[columna] = df[columna].astype(tipo, errors='ignore')
                columnas_convertidas.append(columna)
            except Exception as e:
                print(f"No se pudo convertir la columna '{columna}' a {tipo}: {e}")

    if mostrar_resumen:
        print("Conversión de tipos finalizada.")
        if columnas_convertidas:
            print(f"Columnas convertidas: {columnas_convertidas}")
        else:
            print("No se convirtió ninguna columna.")

        # Mostrar todas las columnas con su tipo de dato final
        print("\nTipos de datos finales por columna:")
        tipos_finales = pd.DataFrame({
            "Columna": df.columns,
            "Tipo de dato": [df[col].dtype for col in df.columns]
        })
        display(tipos_finales)
            
    return df

In [74]:
mapeo_tipos = {
    "age": "Int64",
    "daily_rate": float,
    "hourly_rate": float,
    "monthly_rate": float,
    "training_times_last_year": "Int64",
    "years_with_curr_manager": "Int64",
}

df = convertir_tipos_columnas(df, mapeo_tipos)

Conversión de tipos finalizada.
Columnas convertidas: ['age', 'daily_rate', 'hourly_rate', 'training_times_last_year', 'years_with_curr_manager']

Tipos de datos finales por columna:


,Columna,Tipo de dato
0,age,Int64
1,attrition,object
2,business_travel,object
3,daily_rate,float64
4,department,object
5,distance_from_home,int64
6,education,int64
7,education_field,object
8,environment_satisfaction,int64
9,gender,object


In [75]:
def mapear_columnas_ordinales(df, mapeo_ordinal, mostrar_resumen=True):
    """
    Aplica un mapeo semántico a columnas ordinales de un DataFrame.

    Pasos:
    1. Reemplaza los valores numéricos de las columnas ordinales por etiquetas semánticas.
    2. Permite mostrar un resumen de las columnas mapeadas.

    Parámetros:
    - df: pd.DataFrame a modificar
    - mapeo_ordinal: diccionario donde las claves son nombres de columnas y los valores
      son diccionarios de mapeo {valor_original: valor_nuevo}
    - mostrar_resumen: bool, si True imprime qué columnas se han mapeado

    Retorna:
    - DataFrame con las columnas ordinales mapeadas
    """
    
    df = df.copy()
    columnas_mapeadas = []

    for columna, mapping in mapeo_ordinal.items():
        if columna in df.columns:
            df[columna] = df[columna].map(mapping)
            columnas_mapeadas.append(columna)

    if mostrar_resumen:
        print("🔹 Mapeo de columnas ordinales finalizado.")
        if columnas_mapeadas:
            print(f"Columnas mapeadas: {columnas_mapeadas}")
        else:
            print("No se aplicó mapeo a ninguna columna.")

    return df

In [76]:
def mapear_columnas_ordinales(df, mapeos_columnas, mostrar_resumen=True):
    """
    Aplica mapeo semántico a columnas ordinales del DataFrame según un diccionario proporcionado.

    Parámetros:
    - df: pd.DataFrame a modificar
    - mapeos_columnas: diccionario con claves como nombres de columnas y valores como diccionarios de mapeo.
                       Ejemplo:
                       {
                           "education": {1: "Sin estudios", 2: "Educación básica", ...},
                           "job_level": {1: "Becario", 2: "Junior", ...}
                       }
    - mostrar_resumen: bool, si True imprime resumen de columnas mapeadas y sus valores únicos.

    Retorna:
    - DataFrame con columnas ordinales mapeadas
    """
    df = df.copy()
    columnas_mapeadas = []

    for columna, mapa in mapeos_columnas.items():
        if columna in df.columns:
            df[columna] = df[columna].map(mapa)
            columnas_mapeadas.append(columna)

    if mostrar_resumen:
        print("🔹 Mapeo de columnas ordinales finalizado.")
        print(f"Columnas mapeadas: {columnas_mapeadas}")
        for col in columnas_mapeadas:
            print(f"{col}: {df[col].unique()}")

    return df

In [77]:
satisfaction_map = {
    1: "Nada satisfecho",
    2: "Insatisfecho",
    3: "Satisfecho",
    4: "Muy satisfecho",
}

education_map = {
    1: "Sin estudios",
    2: "Educación básica",
    3: "FP/Bachiller",
    4: "Estudios universitarios",
    5: "Postgrado",
}

job_level_map = {
    1: "Becario",
    2: "Junior",
    3: "Senior",
    4: "Manager",
    5: "Director",
}

satisfaction_cols = [
    "env_satisfaction",
    "job_involvement",
    "job_satisfaction",
    "performance_score",
    "rel_satisfaction",
    "work_life_balance",
]

# Construir diccionario columna → mapa
mapeos_ordinales = {col: satisfaction_map for col in satisfaction_cols}
mapeos_ordinales["education"] = education_map
mapeos_ordinales["job_level"] = job_level_map


df = mapear_columnas_ordinales(df, mapeos_ordinales)

🔹 Mapeo de columnas ordinales finalizado.
Columnas mapeadas: ['job_involvement', 'job_satisfaction', 'work_life_balance', 'education', 'job_level']
job_involvement: ['Satisfecho' 'Insatisfecho' 'Muy satisfecho' 'Nada satisfecho']
job_satisfaction: ['Muy satisfecho' 'Insatisfecho' 'Satisfecho' 'Nada satisfecho' nan]
work_life_balance: ['Nada satisfecho' 'Satisfecho' 'Insatisfecho' 'Muy satisfecho']
education: ['Educación básica' 'Sin estudios' 'Estudios universitarios'
 'FP/Bachiller' 'Postgrado']
job_level: ['Junior' 'Becario' 'Senior' 'Manager' 'Director']


In [ ]:
def imputar_categoricas(df, columnas, umbral_nulos_alto=0.20, umbral_nulos_bajo=0.05,
                        umbral_moda_bajo=0.50, umbral_moda_medio=0.60, umbral_ventaja=0.20,
                        etiqueta_unknown="Unknown"):
    """
    Imputa nulos en variables categóricas siguiendo reglas simples y justificables en EDA.

    Reglas (resumen):
    1) Si el % de nulos es ALTO (> umbral_nulos_alto, por defecto 20%) → imputar con etiqueta_unknown.
       - Motivo: imputar por moda con muchos nulos puede inventar demasiada información.

    2) Si el % de nulos es BAJO (<= umbral_nulos_bajo, por defecto 5%) o MEDIO (entre 5% y 20%):
       - Solo imputamos con la MODA si hay una categoría realmente dominante.
       - Para considerar "dominante" exigimos 2 condiciones:
         a) La moda supera un umbral según el % de nulos:
            - Si nulos BAJOS: moda >= umbral_moda_bajo (50% por defecto)
            - Si nulos MEDIOS: moda >= umbral_moda_medio (60% por defecto)
         b) La moda tiene suficiente ventaja sobre la 2ª categoría:
            - (pct_moda - pct_segunda) >= umbral_ventaja (20 puntos porcentuales por defecto)

       Si no se cumple lo anterior → imputar con etiqueta_unknown.

    Parámetros
    ----------
    df : pandas.DataFrame
        DataFrame de entrada (se modifica y también se devuelve).
    columnas : list[str]
        Lista de columnas categóricas a imputar.
    umbral_nulos_alto : float, default 0.20
        Por encima de este porcentaje de nulos se usa etiqueta_unknown.
    umbral_nulos_bajo : float, default 0.05
        Hasta este porcentaje de nulos se considera "bajo".
    umbral_moda_bajo : float, default 0.50
        Umbral mínimo de la moda si los nulos son bajos.
    umbral_moda_medio : float, default 0.60
        Umbral mínimo de la moda si los nulos son medios (5%-20%).
    umbral_ventaja : float, default 0.20
        Ventaja mínima (en proporción, 0.20 = 20 puntos porcentuales) entre la moda y la 2ª categoría.
    etiqueta_unknown : str, default "Unknown"
        Etiqueta para imputar cuando no se quiere usar la moda.

    Returns
    -------
    pandas.DataFrame
        El mismo DataFrame con los nulos imputados en las columnas indicadas.
    """

    total = len(df)

    for col in columnas:
        print(f"\n📌 Analizando columna: {col}")
        
        # Si la columna no existe, evitamos error y seguimos
        if col not in df.columns:
            print(f"❌ La columna {col} no existe en el DataFrame. Se omite.")
            continue

        nulos = df[col].isnull().sum()
        porcentaje_nulos = nulos / total if total > 0 else 0

        print(f"   → Nulos: {nulos} de {total} ({porcentaje_nulos:.2%})")

        # Caso 1: muchos nulos -> Unknown directamente
        if porcentaje_nulos > umbral_nulos_alto:
            print(
                f"   🔴 Porcentaje de nulos > {umbral_nulos_alto:.0%} "
                f"→ se crea la categoría '{etiqueta_unknown}'"
            )
            df[col] = df[col].fillna(etiqueta_unknown)
            continue
        
        # Calculamos frecuencias (sin nulos) para decidir si hay moda dominante
        valores = df[col].value_counts(dropna=True)

        if len(valores) == 0:
            # No hay valores no nulos para decidir moda (columna vacía o todo nulo)
            print(
                f"   🔴 No hay valores no nulos para decidir moda "
                f"→ se crea la categoría '{etiqueta_unknown}'"
            )
            df[col] = df[col].fillna(etiqueta_unknown)
            continue

        # Frecuencia y porcentaje de la moda
        primero = valores.iloc[0]
        pct_primero = primero / total # proporción sobre el total de filas

        # Frecuencia y porcentaje de la segunda categoría (si existe)
        if len(valores) > 1:
            segundo = valores.iloc[1]
            pct_segundo = segundo / total
        else:
            pct_segundo = 0.0

        ventaja = pct_primero - pct_segundo # ventaja de la moda frente a la 2ª (en proporción)

        print(f"   → Moda: {valores.index[0]} ({pct_primero:.2%})")
        print(f"   → 2ª categoría: {valores.index[1] if len(valores) > 1 else 'No existe'} ({pct_segundo:.2%})")
        print(f"   → Ventaja de la moda: {ventaja:.2%}")

        # Umbral de moda depende de si el % de nulos es bajo o medio
        if porcentaje_nulos <= umbral_nulos_bajo:
            umbral_moda = umbral_moda_bajo
            print(
                f"   → Nulos bajos (≤ {umbral_nulos_bajo:.0%}), "
                f"umbral de moda requerido: {umbral_moda:.0%}"
            )
        else:
            umbral_moda = umbral_moda_medio
            print(
                f"   → Nulos medios (> {umbral_nulos_bajo:.0%} y ≤ {umbral_nulos_alto:.0%}), "
                f"umbral de moda requerido: {umbral_moda:.0%}"
            )
        
        # Caso 2: imputar por moda solo si es dominante y con ventaja suficiente
        if (pct_primero >= umbral_moda) and (ventaja >= umbral_ventaja):
            moda = df[col].mode(dropna=True)[0]
            print(
                f"   🟢 La moda es dominante y con ventaja suficiente "
                f"(≥ {umbral_ventaja:.0%}) → se imputan nulos con '{moda}'"
            )
            df[col] = df[col].fillna(moda)
        else:
            print(
                f"   🟡 La moda NO es lo suficientemente dominante "
                f"→ se crea la categoría '{etiqueta_unknown}'"
            )
            df[col] = df[col].fillna(etiqueta_unknown)

    return df

In [ ]:
cols_cat = df.select_dtypes(include=["object", "category"]).columns.tolist()
cols_cat = [c for c in cols_cat if c.lower() != "attrition"]  # excluimos attrition

# normaliza falsos nulos solo en esas columnas
df[cols_cat] = df[cols_cat].replace(r'^\s*$', pd.NA, regex=True)

# imputación
df = imputar_categoricas(df, cols_cat)

In [ ]:
def imputar_numericas(df, columnas, umbral_nulos_bajo=0.05, umbral_nulos_alto=0.20,
                      n_neighbors=5, crear_indicador_missing=True, usar_knn_en_alto=False):
    """
    Imputa nulos en variables numéricas siguiendo reglas simples y justificables en EDA,
    usando mediana (robusta) cuando el % de nulos es bajo y KNNImputer cuando es moderado.

    Reglas (resumen):
    1) Si el % de nulos es BAJO (<= umbral_nulos_bajo, por defecto 5%) → imputar con mediana.
       - Motivo: con pocos nulos, la mediana es estable y minimiza el efecto de outliers.

    2) Si el % de nulos es MODERADO (> umbral_nulos_bajo y <= umbral_nulos_alto, por defecto 5%-20%)
       → imputar con KNNImputer (por defecto k=5) usando registros similares.
       - Motivo: con más nulos, KNN puede aprovechar el “contexto” de otras variables numéricas.

    3) Si el % de nulos es ALTO (> umbral_nulos_alto, por defecto 20%):
       - (Opcional) crear una variable indicadora de missingness: col + "_missing"
       - Imputar con mediana por defecto (más estable). Si quieres, puedes activar KNN también en alto
         con usar_knn_en_alto=True.

    IMPORTANTE SOBRE KNN + ESCALADO:
    KNN funciona con distancias entre filas. Si las columnas numéricas están en escalas distintas
    (por ejemplo, una en 0-10 y otra en 0-10.000), la columna de rango grande dominaría la distancia.
    Por eso hacemos:
      - Escalado (StandardScaler) → KNNImputer → desescalado
    Así todas las columnas “pesan” parecido al calcular similitud.

    Parámetros
    ----------
    df : pandas.DataFrame
        DataFrame de entrada (se modifica y también se devuelve).
    columnas : list[str]
        Lista de columnas numéricas a imputar.
    umbral_nulos_bajo : float, default 0.05
        Hasta este porcentaje de nulos se considera "bajo".
    umbral_nulos_alto : float, default 0.20
        Por encima de este porcentaje de nulos se considera "alto".
    n_neighbors : int, default 5
        Número de vecinos (k) para KNNImputer.
    crear_indicador_missing : bool, default True
        Si True, cuando el % de nulos es alto se crea una columna col+"_missing" (0/1).
    usar_knn_en_alto : bool, default False
        Si True, en % de nulos alto también se usa KNN (con escalado). Si False, se usa mediana.

    Returns
    -------
    pandas.DataFrame
        El mismo DataFrame con los nulos imputados en las columnas indicadas.
    """

    total = len(df)

    # Nos aseguramos de trabajar solo con columnas que existen
    columnas_validas = [c for c in columnas if c in df.columns]
    columnas_no_encontradas = [c for c in columnas if c not in df.columns]
    for c in columnas_no_encontradas:
        print(f"{c} no existe en el DataFrame. Se omite.")

    # También nos aseguramos de que sean numéricas (por si te cuelas en la lista)
    cols_num_df = df.select_dtypes(include=[np.number]).columns.tolist()
    columnas_validas = [c for c in columnas_validas if c in cols_num_df]

    # Si no queda ninguna, salimos sin romper nada
    if len(columnas_validas) == 0:
        print("No hay columnas numéricas válidas para imputar.")
        return df

    # Preparamos imputador de mediana (lo reutilizamos)
    imputer_mediana = SimpleImputer(strategy="median")

    # Para KNN con escalado, necesitamos un “bloque” numérico:
    # Usamos TODAS las numéricas del DF, porque KNN se beneficia de más contexto.
    # (Esto NO significa que imputemos todas: solo guardamos de vuelta las columnas objetivo.)
    cols_num_contexto = cols_num_df

    for col in columnas_validas:
        print(f"\n📌 Analizando columna numérica: {col}")

        nulos = df[col].isnull().sum()
        porcentaje_nulos = nulos / total if total > 0 else 0
        print(f"   → Nulos: {nulos} de {total} ({porcentaje_nulos:.2%})")

        # CASO 1: % de nulos bajo -> mediana
        if porcentaje_nulos <= umbral_nulos_bajo:
            print(
                f"   🟢 Nulos bajos (≤ {umbral_nulos_bajo:.0%}) "
                f"→ imputación con MEDIANA (SimpleImputer)"
            )
            df[[col]] = imputer_mediana.fit_transform(df[[col]])
            continue

        # CASO 2: % de nulos moderado -> KNN con escalado
        if porcentaje_nulos <= umbral_nulos_alto:
            print(
                f"   🟡 Nulos moderados (> {umbral_nulos_bajo:.0%} y ≤ {umbral_nulos_alto:.0%}) "
                f"→ imputación con KNN (k={n_neighbors}) + ESCALADO"
            )

            # 1) Cogemos el bloque numérico completo (contexto)
            X = df[cols_num_contexto].copy()

            # 2) Escalamos (standardization): (x - media) / desviación
            #    Esto hace comparables las columnas para calcular distancias.
            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X)

            # 3) Imputamos en el espacio escalado
            knn = KNNImputer(n_neighbors=n_neighbors)
            X_imputed_scaled = knn.fit_transform(X_scaled)

            # 4) Desescalamos para volver a las unidades originales
            X_imputed = scaler.inverse_transform(X_imputed_scaled)

            # 5) Volvemos a DataFrame para poder asignar por columnas
            X_imputed = pd.DataFrame(X_imputed, columns=cols_num_contexto, index=df.index)

            # 6) Solo guardamos la columna objetivo (para no tocar otras numéricas fuera de tu lista)
            df[col] = X_imputed[col]

            print(f"   ✅ {col} imputada con KNN + escalado (solo se asigna esta columna).")
            continue

        # CASO 3: % de nulos alto
        print(
            f"   🔴 Nulos altos (> {umbral_nulos_alto:.0%}) "
            f"→ se considera missingness + imputación robusta"
        )

        if crear_indicador_missing:
            indicador = f"{col}_missing"
            # 1 si era nulo, 0 si no
            df[indicador] = df[col].isnull().astype(int)
            print(f"   → Se crea indicador de missingness: {indicador} (1=nulo, 0=no nulo)")

        if usar_knn_en_alto:
            print(
                f"   🟠 usar_knn_en_alto=True "
                f"→ imputación con KNN (k={n_neighbors}) + ESCALADO"
            )

            X = df[cols_num_contexto].copy()
            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X)

            knn = KNNImputer(n_neighbors=n_neighbors)
            X_imputed_scaled = knn.fit_transform(X_scaled)

            X_imputed = scaler.inverse_transform(X_imputed_scaled)
            X_imputed = pd.DataFrame(X_imputed, columns=cols_num_contexto, index=df.index)

            df[col] = X_imputed[col]
            print(f"   ✅ {col} imputada con KNN + escalado (solo se asigna esta columna).")

        else:
            print("   🟠 Se imputa con MEDIANA (SimpleImputer) por estabilidad.")
            df[[col]] = imputer_mediana.fit_transform(df[[col]])

    return df

In [ ]:
def limpieza_general(
    df,
    id_columna='employee_number',
    mapeo_tipos=None,
    mapeos_texto=None,
    mapeos_ordinales=None,
    columnas_categoricas_nulos=None,
    columnas_numericas_nulos=None,
    mostrar_resumen=True
):
    """
    Función general de limpieza que llama a las funciones individuales previamente definidas.

    Pasos incluidos:
    1. Normalización de nombres de columnas (snake_case)
    2. Establecer columna ID como índice y renombrarla a 'id'
    3. Eliminación de filas duplicadas
    4. Eliminación de columnas sin aporte analítico
    5. Conversión de tipos de columnas
    6. Normalización de columnas de texto con mapeos opcionales
    7. Mapeo de columnas ordinales
    8. Imputación de nulos en columnas categóricas
    9. Imputación de nulos en columnas numéricas

    Retorna:
    - DataFrame limpio listo para el análisis
    """

    df = df.copy()

    # 1. Normalizar nombres de columnas
    df.columns = normalizar_nombres_columnas(df.columns.tolist(), mostrar_resumen=mostrar_resumen)

    # 2. Establecer ID como índice y renombrar a 'id'
    df = usar_columna_como_indice(df, columna_original=id_columna, indice='id')

    # 3. Eliminar duplicados
    df = eliminar_filas_duplicadas(df)

    # 4. Eliminar columnas sin aporte analítico
    df = eliminar_columnas_sin_aporte_analitico(df)

    # 5. Conversión de tipos de columnas
    if mapeo_tipos:
        df = convertir_tipos_columnas(df, mapeo_tipos, mostrar_resumen=mostrar_resumen)

    # 6. Normalización de columnas de texto
    if mapeos_texto:
        df = normalizar_columnas_texto(df, mapeos_reemplazo=mapeos_texto, mostrar_resumen=mostrar_resumen)

    # 7. Mapeo de columnas ordinales
    if mapeos_ordinales:
        df = mapear_columnas_ordinales(df, mapeos_ordinales, mostrar_resumen=mostrar_resumen)

    # 8. Imputación de nulos en columnas categóricas
    if columnas_categoricas_nulos:
        df = imputar_categoricas(df, columnas_categoricas_nulos)

    # 9. Imputación de nulos en columnas numéricas
    if columnas_numericas_nulos:
        df = imputar_numericas(df, columnas_numericas_nulos)

    if mostrar_resumen:
        print("\n🟢 Limpieza general completada.")

    return df

In [ ]:
df.to_csv('../data/processed/hr_processed.csv')